## RNN(순환신경망)
- Recurrent(순환하다 = 반복해서 되돌아감) Neural Network
- feed forward 신경망은 고정 길이로 인해 시계열 데이터 학습에 불리 -> 맥락의 정보를 기억하는 순환 신경망이 시계열 데이터 처리에 적합

In [2]:
# RNN 구현

import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None #cache = 역전파 중간 데이터 저장

    def forward(self, x, h_prev): #h_prev = 이전 h값
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next):
        Wx, Wh, b = self.param
        x, h_prev, h_next = self.cache

        dt = dh_next * (1-h_next**2)
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(x.T, dt)
        dWx = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

In [5]:
# TimeRNN(시계열 데이터를 한꺼번에 처리하는 계층) 구현

class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful #은닉 상태 인계 여부(T/F) -> T이면 마지막 RNN 계층 은닉상태 이용, F이면 영행렬로 초기화 됨
        
    def set_state(self,h):
        self.h = h

    def reset_state(self):
        self.h = None

    def forward(self, xs): #xs = T개 분량의 시계열 데이터를 하나로 모은 것
        Wx, Wh, b = self.params 
        N, T, D = xs.shape #N = 미니배치 크기, D = 차원 수
        D, H = Wx.shape

        self.layers = []
        hs = np.empty((N,T,H), dtype='f') #f = 실수

        if not self.stateful or self.h is None:
            self.h = np.zeros((N,H), dtpe='f')
        
        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:,t,:], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)

        return hs
    
    def backward(self, dhs):
        Wx,Wh,b = self.params
        N,T,H = dhs.shape
        D, H = Wx.shape

        dxs = np.empty((N,T,D), dtype='f') #각 시간 단계에서의 입력에 대한 기울기를 저장
        dh = 0
        grads = [0,0,0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:,t,:] + dh) #합산된 기울기(현재 시간 단계의 역전파를 수행하고, 이전 시간 단계에서의 은닉 상태의 기울기 dh와 합산하여 반환)
            dxs[:,t,:] = dx # [N,T,D]

            for i, grad in enumerate(layer.grads):
                grads[i] += grad
            self.dh = dh

        return dxs